This Notebook handles the code for the classification model

In [127]:
%pip install torch torchvision torchaudio
%pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [128]:
%pip install molvs

Note: you may need to restart the kernel to use updated packages.


In [129]:
import os
import pickle
import torch
from torch import nn
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool
import numpy as np
from rdkit import Chem
from torch_geometric.utils import from_smiles
import pandas as pd
from molvs import standardize_smiles
from sklearn.ensemble import RandomForestClassifier
CONFIG = {
    'data_dir': './processed_tox21',
    'hidden_channels': 128,
    'num_layers': 3,
    'dropout': 0.2,
    'batch_size': 64,
    'lr': 1e-3,
    'weight_decay': 0,
    'epochs': 50,
    'patience': 8,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Automatically detects if you have a GPU
print(f"Using device: {CONFIG['device']}")


Using device: cpu


Load data from preprocessing

In [130]:
def load_split(name):
    path = os.path.join(CONFIG['data_dir'], f'tox21_{name}.pkl')
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data


data_train = load_split('train')
data_validation = load_split('validation')
data_test = load_split('test')

print(f"Train: {len(data_train['smiles'])} | Validation: {len(data_validation['smiles'])} | Test: {len(data_test['smiles'])}")

Train: 6258 | Validation: 782 | Test: 783


Simplify the labels

In [131]:
data_train['labels'].shape

(6258, 12)

In [132]:
data_train['labels']

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., nan, nan, ...,  1., nan, nan],
       [ 0.,  0.,  0., ..., nan, nan,  0.],
       ...,
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
# cols = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12']
y_train = pd.DataFrame(data_train['labels']  ).fillna(0)
y_test = pd.DataFrame(data_test['labels'] ).fillna(0)

val_test = pd.DataFrame(data_validation['labels']).fillna(0)

# extract labels
# y_train_label = pd.DataFrame(y_train.max(axis=1), columns=['toxic'])
# y_test_label = pd.DataFrame(y_test.max(axis=1), columns=['toxic'])
val_test_label =pd.DataFrame(val_test.max(axis=1), columns=['toxic']) 

In [ ]:
# y_train_label.value_counts()

In [ ]:
smiles_train = data_train['smiles']
smiles_test = data_test['smiles']

Convert SMILES into ECFP

In [ ]:
# Code from https://drzinph.com/ECFP-fingerprints-in-python-part-3/
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs
class ECFP:
    def __init__(self, smiles):
        self.mols = [Chem.MolFromSmiles(i) for i in smiles]
        self.smiles = smiles

    def mol2fp(self, mol, radius = 3):
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius = radius)
        # fp = AllChem.MorganGenerator(mol, radius = radius)
        array = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, array)
        return array

    def compute_ECFP(self):
        bit_headers = ['bit' + str(i) for i in range(2048)]
        arr = np.empty((0,2048), int).astype(int)
        for i in self.mols:
            fp = self.mol2fp(i)
            arr = np.vstack((arr, fp))
        df_ECFP = pd.DataFrame(np.asarray(arr).astype(int),columns=bit_headers)
        df_ECFP.insert(loc=0, column='smiles', value=self.smiles)
        return df_ECFP

In [153]:
smiles = [standardize_smiles(i) for i in smiles_train] 
sm = [standardize_smiles(i) for i in smiles_test] 

In [154]:
# train set
ECFP_descriptor_train = ECFP(smiles)        # create your ECFP object and provide smiles
x_train_ecfp = ECFP_descriptor_train.compute_ECFP() # compute

# test set
ECFP_descriptor_test = ECFP(sm)        # create your ECFP object and provide smiles
x_test_ecfp = ECFP_descriptor_test.compute_ECFP() # compute

In [155]:
train_data = pd.concat([x_train_ecfp, y_train], axis=1)
train_data.head()

,smiles,bit0,bit1,bit2,bit3,bit4,bit5,bit6,bit7,bit8,...,2,3,4,5,6,7,8,9,10,11
0,COC(=O)[C@H]1CC[C@H](C(=O)OC)CC1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,O=S1(=O)CC(Cl)(Cl)C(Cl)(Cl)C1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2,Cc1cc(C)c(NC(=O)C[C@H](CC(=O)[O-])c2cccc3ccccc...,0,1,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cc1noc(NS(=O)(=O)c2ccc(N)cc2)c1C,0,0,0,0,1,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CC(C)(C)NC[C@H](O)COc1nsnc1N1CCOCC1,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:

x_train_ecfp.iloc[:, 1:]

,bit0,bit1,bit2,bit3,bit4,bit5,bit6,bit7,bit8,bit9,...,bit2038,bit2039,bit2040,bit2041,bit2042,bit2043,bit2044,bit2045,bit2046,bit2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6253,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6254,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Random Forest

In [296]:
from sklearn.multioutput import MultiOutputClassifier
rf = RandomForestClassifier(
    n_estimators=1000,
    class_weight='balanced',
    random_state=42
)
cf = MultiOutputClassifier( rf)
cf.fit(x_train_ecfp.iloc[:, 1:], y_train)

y_pred = cf.predict(x_test_ecfp.iloc[:, 1:])
y_pred_prob = np.column_stack([clf.predict_proba(x_test_ecfp.iloc[:, 1:])[:, 1] for clf in cf.estimators_])

In [330]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, precision_score, recall_score, f1_score, balanced_accuracy_score
y_true = y_test
y_pred = pd.DataFrame(y_pred)
y_prob_df = pd.DataFrame(y_pred_prob)
metrics = []
for i in range(12):
    yt = y_true[i]
    yp = y_pred[i]
    yprob = y_prob_df[i]
    metrics.append(
        (
            roc_auc_score(yt, yprob),
            accuracy_score(yt, yp),
            precision_score(yt, yp),
            recall_score(yt, yp),
            f1_score(yt, yp),
        )
    )  



In [331]:
rf_metrics = pd.DataFrame(metrics, columns=[
     "ROC_AUC", "Accuracy", "Precision", "Recall", "F1-score"
])
rf_metrics

,ROC_AUC,Accuracy,Precision,Recall,F1-score
0,0.733564,0.961686,0.545455,0.375000,0.444444
1,0.846464,0.983397,0.764706,0.590909,0.666667
2,0.875222,0.920817,0.727273,0.390244,0.507937
3,0.766944,0.966794,0.857143,0.193548,0.315789
4,0.697194,0.896552,0.555556,0.235294,0.330579
5,0.801917,0.966794,0.720000,0.486486,0.580645
6,0.805917,0.982120,0.500000,0.071429,0.125000
7,0.721384,0.853129,0.416667,0.090090,0.148148
8,0.834014,0.961686,1.000000,0.117647,0.210526
9,0.738922,0.943806,0.363636,0.097561,0.153846


In [332]:
rf_metrics.mean()

ROC_AUC      0.791187
Accuracy     0.939974
Precision    0.657729
Recall       0.250197
F1-score     0.334193
dtype: float64

# GNN VS Base Classification Model

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, model_type='GCN', num_layers=3, dropout=0.2):
        super().__init__()
        self.model_type = model_type
        self.convs = nn.ModuleList()
        if model_type == 'GCN':
            # Aggregates information from neighboring atoms
            self.convs.append(GCNConv(in_channels, hidden_channels))
            for _ in range(num_layers-1):
                self.convs.append(GCNConv(hidden_channels, hidden_channels))
        elif model_type == 'GAT':
            # Learns to weight each neighbour
            self.convs.append(GATConv(in_channels, hidden_channels, heads=4, concat=False))
            for _ in range(num_layers-1):
                self.convs.append(GATConv(hidden_channels, hidden_channels, heads=4, concat=False))
        else:
            raise ValueError("model_type must be 'GCN' or 'GAT'")
        
        self.dropout = dropout
        self.lin = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels//2, out_channels)
        )

    def forward(self, x, edge_index, batch):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = torch.relu(x)
            x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = global_mean_pool(x, batch)
        return self.lin(x)
    
    def forward_with_attention(self, x, edge_index, batch):
        # Only returns attention weights for the first GAT layer
        if self.model_type != 'GAT':
            raise ValueError("forward_with_attention is only available for GAT models")
        
        x = x.float()
        x, (attn_edge_index, attn_values) = self.convs[0](x, edge_index, return_attention_weights=True)
        x = torch.relu(x)
        x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        
        for conv in self.convs[1:]:
            x = conv(x, edge_index)
            x = torch.relu(x)
            x = nn.functional.dropout(x, p=self.dropout, training=self.training)

        x = global_mean_pool(x, batch)
        out = self.lin(x)
        return out, attn_edge_index, attn_values

In [ ]:
def build_graph(smi, labels):
    try:
        data = from_smiles(smi)
        labels = np.array(labels, dtype=float).reshape(-1)
        data.y = torch.tensor(labels, dtype=torch.float).unsqueeze(0)
        return data
    except Exception:
        return None


def make_dataset(smiles_list, label_matrix):
    dataset = []
    for smi, lbl in zip(smiles_list, label_matrix):
        g = build_graph(smi, lbl)
        if g is not None:
            dataset.append(g)
    
    return dataset

In [ ]:
train_dataset = make_dataset(data_train['smiles'], data_train['labels'])
validation_dataset = make_dataset(data_validation['smiles'], data_validation['labels'])
test_dataset = make_dataset(data_test['smiles'], data_test['labels'])

print(f"Graphs Data")
print(f"Train: {len(train_dataset)} | Validation: {len(validation_dataset)} | Test: {len(test_dataset)}")

Graphs Data
Train: 6258 | Validation: 782 | Test: 783


In [ ]:
def compute_task_auc(model, loader, device):
    model.eval()
    all_logits = []
    all_labels = []
    y_pred = []
    with torch.no_grad():
        for data in loader:
            data.x = data.x.to(torch.float)
            data.edge_index = data.edge_index.to(torch.long)
            if hasattr(data, "edge_attr") and data.edge_attr is not None:
                data.edge_attr = data.edge_attr.to(torch.float)
            data.y = data.y.to(torch.float)

            data = data.to(device)

            logits = model(data.x, data.edge_index, data.batch)
            labels = data.y

            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())

    logits = torch.cat(all_logits, dim=0)
    labels = torch.cat(all_labels, dim=0)

    probs = torch.sigmoid(logits).detach().numpy()
    labels = labels.numpy()
    propbs = np.nan_to_num(probs)
    labels = np.nan_to_num(labels)
    aucs = []
    rows = []

    for task_idx in range(labels.shape[1]):
        valid = ~np.isnan(labels[:, task_idx])
        if valid.sum() > 0:
            y_true = labels[valid, task_idx].reshape(-1)
            y_score = probs[valid, task_idx].reshape(-1)
            auc = roc_auc_score(y_true, y_score) if len(np.unique(y_true)) > 1 else np.nan
            y_pred = (y_score >= 0.1).astype(int)
            acc = accuracy_score(y_true, y_pred)
            prec = precision_score(y_true, y_pred, zero_division=0)
            rec = recall_score(y_true, y_pred, zero_division=0)
            f1s = f1_score(y_true, y_pred)
        else:
            auc, acc, prec, rec, f1s = (np.nan, np.nan, np.nan, np.nan, np.nan)

        rows.append((auc, acc, prec, rec, f1s))

    df = pd.DataFrame(rows, columns=[ "ROC_AUC", "Accuracy", "Precision", "Recall", "F1-score" ])
    df = df.apply(lambda col : col.mean() )
    return df

In [339]:
train_loader = DataLoader(train_dataset, batch_size = CONFIG['batch_size'], shuffle = True)
val_loader = DataLoader(validation_dataset, batch_size = CONFIG['batch_size'])
test_loader = DataLoader(test_dataset, batch_size = CONFIG['batch_size'])

In [340]:
sample_graph = train_dataset[0]
in_channels = sample_graph.x.shape[1]
out_channels = sample_graph.y.shape[1]
task_names = data_train['labels_cols']

gcn_model = GNNModel(
    in_channels=in_channels,
    hidden_channels=CONFIG['hidden_channels'],
    out_channels=out_channels,
    model_type='GCN',
    num_layers=CONFIG['num_layers'],
    dropout=CONFIG['dropout']
).to(CONFIG['device'])
gcn_model.load_state_dict(torch.load("best_GCN.pt"))

gat_model = GNNModel(
    in_channels=in_channels,
    hidden_channels=CONFIG['hidden_channels'],
    out_channels=out_channels,
    model_type='GAT',
    num_layers=CONFIG['num_layers'],
    dropout=CONFIG['dropout']
).to(CONFIG['device'])
gat_model.load_state_dict(torch.load("best_GAT.pt"))

gcn_auc = compute_task_auc(gcn_model, test_loader, CONFIG['device'])
gat_auc = compute_task_auc(gat_model, test_loader, CONFIG['device'])

# gcn_auc
pd.DataFrame()
roc_df = pd.DataFrame({
    # 'Task': task_names,
    'GCN': gcn_auc,
    'GAT': gat_auc,
    "RandomForest": rf_metrics.mean()   
})


In [341]:
roc_df.T

,ROC_AUC,Accuracy,Precision,Recall,F1-score
GCN,0.745164,0.917305,0.442777,0.216269,0.226328
GAT,0.753880,0.921456,0.369753,0.185993,0.217178
RandomForest,0.791187,0.939974,0.657729,0.250197,0.334193
